
# Генерация синтетических данных 

Для выполнения ТЗ на позицию Data-analyst предлагается использовать таблицы со следующей структурой данных:

*	Таблица "users" с полями: id, name, email, created_at

*	Таблица "orders" с полями: id, user_id, total_price, created_at

*	Таблица "order_items" с полями: id, order_id, product_name, price, quantity


In [1]:
import pandas as pd
import numpy as np
import random
from faker import Faker
import datetime as dt

### Таблица 'users' с полями: 
* id, 
* name, 
* email, 
* created_at

In [2]:
# Используя метод Faker() из библиотеки faker сгенерируем тестовые данные.
fake = Faker()

n = 5000000 # объявим количество генерируемых значений  (по условию не менее 1 млн. в каждой таблице) 

# Создадим список-генератор, который создаст список names, состоящий из n случайных имён, сгенерированных библиотекой Faker.
names = [fake.name() for i in range(n)]

In [3]:
# Создадим пустой список 'emails' и с помощью цикла заполним его.
# Чтобы значения были уникальными добавим перед  "@"  порядковый номер имени в списке names.
emails = [name.lower().replace(" ", ".") + f"{i}@googlyandex.com" for i, name in enumerate(names)]

In [4]:
# Создадим пустой список 'dates', зададим период времени, в котором будут генерироваться даты,
# с помощью цикла заполним его генерируемыми значениями.

dates = []
date_start = dt.datetime(2023, 1, 1) 
date_end = dt.datetime(2024, 12, 31, 23, 59, 59)

for _ in range(n):
    dates.append(fake.date_time_between(date_start, date_end))

dates.sort()

In [5]:
# Cоздадим пустой DataFrame, куда и будем добавлять сгенерировынные данные. 

df_users = pd.DataFrame({
    'name': names,
    'email': emails,
    'created_at': dates
})

df_users.head()

,name,email,created_at
0,Courtney Le,courtney.le0@googlyandex.com,2023-01-01 00:00:02
1,Michelle Gardner,michelle.gardner1@googlyandex.com,2023-01-01 00:00:53
2,Keith Salas,keith.salas2@googlyandex.com,2023-01-01 00:00:56
3,Alicia Walker,alicia.walker3@googlyandex.com,2023-01-01 00:01:05
4,Margaret Hunt,margaret.hunt4@googlyandex.com,2023-01-01 00:01:06


In [6]:
# Cбросим индекс и выведем его в отдельную колонку.
# Переименуем колонку 'index' в 'id'.
df_users = df_users.reset_index().rename(columns={'index': 'id'})

# Так как мы генерируем наши данные для Postgre SQL, учитывая ограничение первичного и внешнего ключей, а также
# разницу в исчислении в SQL и Python, сдвинем все значения столбца "id" на +1.
df_users['id'] = df_users['id'] + 1


print(df_users.head())
print('----------------------')
print(df_users.shape)
print('----------------------')
print(df_users.dtypes)

   id              name                              email          created_at
0   1       Courtney Le       courtney.le0@googlyandex.com 2023-01-01 00:00:02
1   2  Michelle Gardner  michelle.gardner1@googlyandex.com 2023-01-01 00:00:53
2   3       Keith Salas       keith.salas2@googlyandex.com 2023-01-01 00:00:56
3   4     Alicia Walker     alicia.walker3@googlyandex.com 2023-01-01 00:01:05
4   5     Margaret Hunt     margaret.hunt4@googlyandex.com 2023-01-01 00:01:06
----------------------
(5000000, 4)
----------------------
id                     int64
name                  object
email                 object
created_at    datetime64[ns]
dtype: object


________________________________________________________________________________________

### Таблица "orders" с полями: 
* id, 
* user_id, 
* total_price, 
* created_at

In [7]:
# Таблицу 'orders' сформируем следующим образом:

# 1. ИЗ df_users случайным образом отберём половину строк ['id','created_at']
df_orders = df_users[['id','created_at']].sample((n * 5), replace = True).rename(columns={'id': 'user_id'})

# 2. К каждой дате добавим случайное количество дней из диапазона от 0 до 90
df_orders['created_at'] = pd.to_datetime(df_orders['created_at']) + pd.to_timedelta(np.random.randint(0, 90, size=len(df_orders)), unit='D')
df_orders = df_orders.sort_values(by='created_at', ascending = True).reset_index(drop=True)

# 3. Общую стоимость пока заполним NAN
df_orders['total_price'] = np.nan

# 4. Сбросим индекс и на его основе создадим колонку 'order_id'
df_orders = df_orders.reset_index().rename(columns={'index': 'order_id'}) 
df_orders['order_id'] = df_orders['order_id'] + 1

df_orders.head()

,order_id,user_id,created_at,total_price
0,1,23,2023-01-01 00:04:47,NaN
1,2,32,2023-01-01 00:06:53,NaN
2,3,37,2023-01-01 00:07:48,NaN
3,4,39,2023-01-01 00:08:21,NaN
4,5,50,2023-01-01 00:10:38,NaN


____________________________________________________________________________________________________

По аналогии поступим и с последней таблицей.

Таблица "order_items" с полями: id, order_id, product_name, price, quantity

In [8]:
df_order_items = df_orders[['order_id','total_price']].copy()
df_order_items.head()

,order_id,total_price
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [9]:
product_name=[]
for i in range(n*5):
    product_name.append(fake.bothify(text='Prod.№: ????-########'))

df_order_items['product_name'] = product_name
df_order_items.head()    

,order_id,total_price,product_name
0,1,NaN,Prod.№: bdfH-17128356
1,2,NaN,Prod.№: mPlk-03916075
2,3,NaN,Prod.№: oDRW-69125411
3,4,NaN,Prod.№: udPT-92142590
4,5,NaN,Prod.№: whTG-22369148


In [10]:
quantity=[]
for i in range(n*5):
    quantity.append(random.randrange(1, 100, 1))

df_order_items['quantity'] = quantity

In [11]:
# Создадим список 'prices' и заполним его псевдослучайными целыми числами от 5 до 500 с шагом 5
prices=[]
for i in range(n*5):
     prices.append(random.randrange(5, 500, 5))

df_order_items['price'] = prices

In [12]:
df_order_items['total_price'] = df_order_items.quantity * df_order_items.price

In [13]:
df_order_items = df_order_items.reset_index(drop=True)
df_order_items = df_order_items.reset_index().rename(columns={'index': 'id'})
df_order_items['id'] = df_order_items['id'] + 1
df_order_items.head(2)

,id,order_id,total_price,product_name,quantity,price
0,1,1,1365,Prod.№: bdfH-17128356,39,35
1,2,2,14040,Prod.№: mPlk-03916075,52,270


____________________________________________________________________

In [14]:
# Сохраним DataFrame в файл.csv
# Так как колонка 'id' у нас будет являться первичным ключом, и мы при создании таблицы установим параметры для неё SERIAL PRIMARY KEY, избавимся от колонок 'id' в наших датафреймах.
df_users = df_users[['name', 'email', 'created_at']] 
df_users.to_csv('users.csv', index=False)

In [15]:
df_orders['total_price'] = df_order_items['total_price'].copy()
df_orders = df_orders[['user_id', 'created_at', 'total_price']]
df_orders.to_csv('orders.csv', index=False)

In [16]:
df_order_items = df_order_items[['order_id', 'product_name', 'price', 'quantity']]
df_order_items.to_csv('order_items.csv', index=False)